In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files, drive

def load_csv(method="upload", source=None, concat=False, **read_csv_kwargs):
    """
    Load CSVs in Colab using one of three methods:
      - "upload": upload from local machine
      - "drive": read from Google Drive
      - "web": read from URL(s)

    Args:
        method: "upload" | "drive" | "web"
        source: file path(s) or URL(s); not needed for upload
        concat: if True, combine all CSVs into one DataFrame
        **read_csv_kwargs: passed to pandas.read_csv()

    Returns:
        A DataFrame (if concat=True or one file) or dict of {name: DataFrame}

    Examples:
        df1 = load_csv("upload")

        path = "/content/drive/MyDrive/data/UScomments.csv"
        df2 = load_csv("drive", path)

        url = "https://people.sc.fsu.edu/~jburkardt/data/csv/addresses.csv"
        df3 = load_csv("web", url)
    """
    defaults = {"on_bad_lines": "skip"}
    kwargs = {**defaults, **(read_csv_kwargs or {})}

    method = method.lower()
    dfs = {}

    if method == "upload":
        uploaded = files.upload()
        for name in uploaded.keys():
            dfs[name] = pd.read_csv(name, **kwargs)

    elif method == "drive":
        drive.mount("/content/drive", force_remount=False)
        if isinstance(source, str):
            source = [source]
        for path in source:
            dfs[path.split("/")[-1]] = pd.read_csv(path, **kwargs)

    elif method == "web":
        if isinstance(source, str):
            source = [source]
        for url in source:
            dfs[url.split("/")[-1]] = pd.read_csv(url, **kwargs)

    else:
        raise ValueError("method must be one of: 'upload', 'drive', 'web'")

    if concat:
        return pd.concat(list(dfs.values()), ignore_index=True)
    return dfs if len(dfs) > 1 else next(iter(dfs.values()))

In [ ]:
method = "upload" # can put in upload, drive, or web
# Note if picking drive specify a path and if picking web specify a URL

df1 = load_csv(method)
df1.head()

In [ ]:
def basic_sanity_summary(df: pd.DataFrame, top_n: int = 3) -> pd.DataFrame:
    n = len(df)
    rows = []

    for col in df.columns:
        s = df[col]
        dtype = s.dtype
        nulls = int(s.isna().sum())
        uniques = int(s.nunique(dropna=True))

        blank = 0
        whitespace = 0
        top_values = ""

        if pd.api.types.is_object_dtype(s) or pd.api.types.is_string_dtype(s):
            s_str = s.astype("string")
            blank = int((s_str == "").sum(skipna=True))
            whitespace = int(s_str.str.match(r"^\s+$", na=False).sum())

            vc = s_str.fillna("<NA>").value_counts(dropna=False).head(top_n)
            top_values = "; ".join([f"{idx} ({cnt})" for idx, cnt in vc.items()])
        else:
            vc = s.value_counts(dropna=False).head(top_n)
            top_values = "; ".join([f"{idx} ({cnt})" for idx, cnt in vc.items()])

        rows.append({
            "column": col,
            "dtype": str(dtype),
            "rows": n,
            "nulls": nulls,
            "null_%": round(nulls / n, 4) if n else 0.0,
            "blank_strings": blank,
            "whitespace_only": whitespace,
            "unique_values": uniques,
            "top_values": top_values
        })

    summary = pd.DataFrame(rows)
    summary = summary.sort_values(
        ["null_%", "blank_strings", "whitespace_only"],
        ascending=False
    ).reset_index(drop=True)

    return summary


summary = basic_sanity_summary(df1, top_n=3)
summary

In [ ]:
for c in ["Age", "DTIRatio"]:
    print(c, "dtype:", df1[c].dtype)
    display(df1[c].map(type).value_counts().head(10))

In [20]:
for c in ["Age", "DTIRatio", "Income", "LoanAmount", "InterestRate", "CreditScore", "NumCreditLines", "MonthsEmployed"]:
    if c in df1.columns:
        df1[c] = pd.to_numeric(df1[c], errors="coerce")

In [ ]:
SAFE_NULL_TOKENS = {"", "null", "none", "nan", "n/a"}  # exclude "na" unless you confirm

def clean_blank_like_values(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    obj_cols = df.select_dtypes(include=["object"]).columns

    for col in obj_cols:
        s = df[col]

        # Strip whitespace ONLY for string cells
        df[col] = s.map(lambda x: x.strip() if isinstance(x, str) else x)

        # Replace safe null tokens (case-insensitive) ONLY for string cells
        df[col] = df[col].map(
            lambda x: np.nan if isinstance(x, str) and x.strip().lower() in SAFE_NULL_TOKENS else x
        )

    return df

df1 = clean_blank_like_values(df1)

display(df1.isna().sum().sort_values(ascending=False).head(20))

In [22]:
numeric_cols = df1.select_dtypes(include=["number"]).columns

print("Numeric columns:")
print(numeric_cols.tolist())


Numeric columns:
['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'Default']


In [ ]:
for c in ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'Default']:
    print(c, "dtype:", df1[c].dtype)
    display(df1[c].map(type).value_counts().head(10))

In [ ]:
num_cols = df1.select_dtypes(include=["number"]).columns
null_counts = df1[num_cols].isna().sum().sort_values(ascending=False)

display(null_counts[null_counts > 0])

In [ ]:
duplicates = df1.duplicated().sum()
print("Duplicate rows:", duplicates)

In [ ]:
before = len(df1)
df1 = df1.dropna()
after = len(df1)

print("Rows before:", before)
print("Rows remaining:", after)

In [27]:
# Optional: drop duplicates
df = df1.drop_duplicates()

In [ ]:
OUTPUT_FILE = "loan_default_cleaned.csv"

df.to_csv(OUTPUT_FILE, index=False)

print("Saved cleaned file:", OUTPUT_FILE)


In [ ]:
from google.colab import files
files.download(OUTPUT_FILE)